In [ ]:
!pip install wandb -qqq
import wandb
wandb.login()

     |████████████████████████████████| 1.8MB 5.7MB/s 
     |████████████████████████████████| 81kB 9.3MB/s 
     |████████████████████████████████| 102kB 9.4MB/s 
     |████████████████████████████████| 163kB 21.1MB/s 
     |████████████████████████████████| 133kB 16.7MB/s 
     |████████████████████████████████| 71kB 8.7MB/s 


<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
# цепляю диск
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as f
import torch.nn.utils
from typing import List, Tuple, Dict
from collections import namedtuple
import nltk

import math
import numpy as np
from nltk.translate.bleu_score import corpus_bleu, sentence_bleu, SmoothingFunction
from docopt import docopt
import json

import sys
import time

import pickle

Hypothesis = namedtuple('Hypothesis', ['value', 'score'])

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

<div class='alert alert-block alert-info'>
<b>model_embeddings.py</b>
    <ul>
        <li><i>Нахуя нам вообще Vocab и VocabEntry?</i> А мы тут ахуенно <b>сторим слова.</b> Нам необходимо связять имбеддинги, которые мы выучим, со словами. А вокабы – маппинг слов и индексов, который мы берем из json'ов
        <li><i>Как работает nn.Embeddings?</i> Лукап таблица индекс – имбеддинг.
    </ul>
</div>

In [ ]:
class VocabEntry(object):
    """
    Vocabulary Entry, i.e. structure containing either src or tgt language terms.
    """
    def __init__(self, word2id: dict=None):
        if word2id:
            self.word2id = word2id
        else:
            self.word2id = dict()
            self.word2id['<pad>'] = 0  # Pad Token
            self.word2id['<s>'] = 1    # Start Token
            self.word2id['</s>'] = 2   # End Token
            self.word2id['<unk>'] = 3  # Unknown Token
        
        self.unk_id = self.word2id['<unk>']
        self.id2word = {v: k for k, v in self.word2id.items()}
        
    def __getitem__(self, word: str) -> int:
        return self.word2id.get(word, self.unk_id)
    
    def __contains__(self, word: str) -> bool:
        return word in self.word2id
    
    def __len__(self) -> int:
        return len(self.word2id)
    
    def id2word(self, idx) -> str:
        return self.id2word[idx]
    
    def words2indices(self, sentences):
        return [self[word] for word in sentences]\
            if type(sentences[0]) == str\
            else [[self[word] for word in sentence] for sentence in sentences]
    
    def indices2words(self, indices: List[int]) -> List[str]:
        return [self.id2word[index] for index in indices]
    
    # Convert list of sentences (words) into tensor with necessary padding for shorter sentences
    def to_input_tensor(self, sents: List[List[str]], device: torch.device) -> torch.Tensor:
        sents = self.words2indices(sents)
        
        # (batch, sentence_len)
        sents_padded = []
        max_length = len(max(sents, key=len))
        for sent in sents:
            sents_padded.append(sent + [self.word2id['<pad>']] * (max_length - len(sent)))
        
        return torch.tensor(sents_padded, dtype=torch.long, device=device).permute(1, 0)
    
    # ЛЕНЬ СУКА
    def __repr__(self):
        return 'Vocabulary[size=%d]' % len(self)
    
    def __setitem__(self, key, value):
        raise ValueError('vocabulary is readonly')
        
    def add(self, word):
        if word not in self:
            wid = self.word2id[word] = len(self)
            self.id2word[wid] = word
            return wid
        else:
            return self[word]
        
    @staticmethod
    def from_corpus(corpus, size, freq_cutoff=2):
        vocab_entry = VocabEntry()
        word_freq = Counter(chain(*corpus))
        valid_words = [w for w, v in word_freq.items() if v >= freq_cutoff]
        print('number of word types: {}, number of word types w/ frequency >= {}: {}'
              .format(len(word_freq), freq_cutoff, len(valid_words)))
        top_k_words = sorted(valid_words, key=lambda w: word_freq[w], reverse=True)[:size]
        for word in top_k_words:
            vocab_entry.add(word)
        return vocab_entry
    
    
class Vocab(object):
    
    def __init__(self, src_vocab: VocabEntry, tgt_vocab: VocabEntry):
        self.src = src_vocab
        self.tgt = tgt_vocab
    
    # ЛЕНЬ СУКА
    @staticmethod
    def build(src_sents, tgt_sents, vocab_size, freq_cutoff) -> 'Vocab':
        assert len(src_sents) == len(tgt_sents)

        print('initialize source vocabulary ..')
        src = VocabEntry.from_corpus(src_sents, vocab_size, freq_cutoff)

        print('initialize target vocabulary ..')
        tgt = VocabEntry.from_corpus(tgt_sents, vocab_size, freq_cutoff)

        return Vocab(src, tgt)

    
    def save(self, file_path):
        json.dump(dict(src_word2id=self.src.word2id, tgt_word2id=self.tgt.word2id), open(file_path, 'w'), indent=2)

    
    @staticmethod
    def load(file_path):
        entry = json.load(open(file_path, 'r'))
        src_word2id = entry['src_word2id']
        tgt_word2id = entry['tgt_word2id']

        return Vocab(VocabEntry(src_word2id), VocabEntry(tgt_word2id))
    
    def __repr__(self):
        return 'Vocab(source %d words, target %d words)' % (len(self.src), len(self.tgt))

In [ ]:
class ModelEmbeddings(nn.Module):
    
    def __init__(self, word_embed_size: int, vocab: Vocab):
        super(ModelEmbeddings, self).__init__()
        self.word_embed_size = word_embed_size
        
        src_pad_token_idx = vocab.src['<pad>']
        tgt_pad_token_idx = vocab.tgt['<pad>']
        
        # A simple lookup table that stores embeddings of a fixed dictionary and size.
        # This module is often used to store word embeddings and retrieve them using indices.
        # The input to the module is a list of indices, and the output is the corresponding word embeddings.
        # padding_idx (int, optional) – If given,
        # pads the output with the embedding vector at padding_idx (initialized to zeros)
        # whenever it encounters the index.
        self.source = nn.Embedding(num_embeddings=len(vocab.src),
                                   embedding_dim=word_embed_size,
                                   padding_idx=src_pad_token_idx)
        self.target = nn.Embedding(len(vocab.tgt), word_embed_size, tgt_pad_token_idx)

<div class='alert alert-block alert-info'>
<b>nmt_model.py</b>
    <ul>
        <li><i>Что происходит в форварде?</i> Тупа от начала и до конца: берем слова, переводим, считаем лосс
        <li><i>Зачем нам маски?</i> Для инкодера – шобы паддинги не портили нам аттеншн. Для декодера – шобы не считать лосс для паддингов.
        <li>Бимсерч
        <li>Девайс, загрузка и сохранение модели
    </ul>
</div>

In [ ]:
class NMT(nn.Module):
    
    def __init__(self, word_embed_size: int, hidden_size: int, vocab: Vocab, dropout_rate: float = 0.2):
        super(NMT, self).__init__()
        self.vocab = vocab
        self.hidden_size = hidden_size
        self.dropout_rate = dropout_rate
        
        self.model_embeddings = ModelEmbeddings(word_embed_size, vocab)
        self.encoder = nn.LSTM(word_embed_size, hidden_size, bidirectional=True)
        self.hidden_projection = nn.Linear(2 * hidden_size, hidden_size, bias=False)
        self.cell_projection = nn.Linear(2 * hidden_size, hidden_size, bias=False)
        
        self.decoder = nn.LSTMCell(word_embed_size + hidden_size, hidden_size)
        self.att_projection = nn.Linear(2 * hidden_size, hidden_size, bias=False)
        self.combined_output_projection = nn.Linear(3 * hidden_size, hidden_size, bias=False)
        self.dropout = nn.Dropout(dropout_rate)
        
        self.target_vocab_projection = nn.Linear(hidden_size, len(vocab.tgt), bias=False)
        
    @property
    def device(self) -> torch.device:
        # nn.Embedding.weight
        # the learnable weights of the module of shape (src_vocab_size, word_embed_size) initialized from N(0,1)
        return self.model_embeddings.source.weight.device
    
    # Тупа от начала и до конца: берем слова, переводим, считаем лосс
    def forward(self, source: List[List[str]], target: List[List[str]]) -> torch.Tensor:
        source_lengths = [len(sentence) for sentence in source]
        
        source_padded = self.vocab.src.to_input_tensor(source, device=self.device)
        target_padded = self.vocab.tgt.to_input_tensor(target, device=self.device)

        enc_hiddens, dec_init_state = self.encode(source_padded, source_lengths)
        enc_masks = self.generate_sent_masks(enc_hiddens, source_lengths)
        # (tgt_sentence_len, batch, hidden)
        combined_outputs = self.decode(enc_hiddens, dec_init_state, target_padded, enc_masks)
        
        # predicted log(probability)
        # (tgt_sentence_len, batch, tgt_vocab_size)
        P = f.log_softmax(self.target_vocab_projection(combined_outputs), dim=-1)
        # slice P with indices of gold words
        # (tgt_sentence_len, batch, tgt_vocab_size)[(tgt_sentence_length, batch, 1)] = (tgt_sentence_len, batch)
        # чтобы не слайсить из словаря <START> вырезаем его [1:]
        target_gold_words_log_prob = torch.gather(P, dim=2, index=target_padded[1:].unsqueeze(-1)).squeeze(-1)
        # zero out probabilities for <pad> tokens
        target_masks = (target_padded != self.vocab.tgt['<pad>']).float()
        target_gold_words_log_prob = target_gold_words_log_prob * target_masks[1:]
        
        return target_gold_words_log_prob.sum(0)
        
    # Apply the encoder to source sentences to obtain encoder hidden states.
    # Additionally, take the final states of the encoder and project them to obtain initial states for decoder.
    def encode(self, source_padded: torch.Tensor, source_lengths: List[int]) -> \
            Tuple[torch.Tensor, Tuple[torch.Tensor, torch.Tensor]]:
        # GET WORD EMBEDDINGS
        # sentences are lists of indices of words
        # (src_sentence_length, batch) –> (src_sentence_length, batch, word_emb_size)
        source_emb = self.model_embeddings.source(source_padded)
        # packs a Tensor containing padded sequences of variable length.
        source_packed = nn.utils.rnn.pack_padded_sequence(source_emb, source_lengths)
        
        # GET ALL HIDDEN STATES AND FINAL HIDDEN STATE, CELL STATE
        enc_hiddens, (hidden_state, cell_state) = self.encoder(source_packed)
        # Pads a packed batch of variable length sequences.
        # It is an inverse operation to pack_padded_sequence().
        # The returned Tensor’s data will be of size T x B x *, where T is the length of the longest
        # (batch, src_sentence_length, hidden_size)
        enc_hiddens, lens_unpacked = nn.utils.rnn.pad_packed_sequence(enc_hiddens, batch_first=True)
        
        # INITIALIZE DECODER INITIAL STATES
        # first, concatenate hidden_forward and hidden_backward
        # (forward + backward = 2, batch, hidden_size) –> (batch, 2 * hidden_size)
        # первый для бэкварда – последний для форварда, последний для форварда – первый для бэкварда
        # то есть это последние хидден стейты, t = seq_len
        hidden_concat = torch.cat((hidden_state[0], hidden_state[1]), dim=1)
        dec_init_hidden = self.hidden_projection(hidden_concat)
        
        cell_concat = torch.cat((cell_state[0], cell_state[1]), dim=1)
        dec_init_cell = self.cell_projection(cell_concat)
        
        return enc_hiddens, (dec_init_hidden, dec_init_cell)
    
    def decode(self,
               enc_hiddens: torch.Tensor,
               dec_init_state: Tuple[torch.Tensor, torch.Tensor],
               target_padded: torch.Tensor,
               enc_masks: torch.Tensor) -> torch.Tensor:
        
        hidden_state, cell_state = dec_init_state

        # (batch, src_sentence_length, 2 * hidden_size) –> (batch, src_sentence_length, hidden_size)
        enc_hiddens_proj = self.att_projection(enc_hiddens)
        
        # у того, что есть результат нашей модели, мы чопаем <END>
        # (tgt_sentence_length, batch) –> (tgt_sentence_length, batch, word_emb_size)
        target_emb = self.model_embeddings.target(target_padded[:-1])  # ???
        # initialize o_0
        # (batch, hidden_size)
        combined_output = torch.zeros(target_emb.size(1), self.hidden_size, device=self.device)
        combined_outputs = []
        # iterate over words in sentences
        for tgt_emb in torch.split(target_emb, 1):
            # [(1, batch, word_emb_size), (batch, hidden_size)]
            tgt_emb = torch.cat((tgt_emb.squeeze(0), combined_output), 1)
            # (batch, hidden_size)
            (hidden_state, cell_state) = self.decoder(tgt_emb, (hidden_state, cell_state))
            
            # ATTENTION
            # If input is a (b×n×m) tensor, mat2 is a (b×m×p) tensor, out will be a (b×n×p) tensor.
            # (batch, src_sentence_len, hidden) x (batch, hidden, 1) = (batch, src_sentence_length)
            e_t = torch.bmm(enc_hiddens_proj, hidden_state.unsqueeze(2)).squeeze(2)
            
            # set e_t to -inf where the word is pad in order to make softmax equal 0
            if enc_masks is not None:
                e_t.masked_fill_(enc_masks.bool(), -float('inf'))
                
            alpha_t = f.softmax(e_t, 1)
            # <(batch, 1, src_sentence_len), (batch, src_sentence_len, 2 * hidden)> = (batch, 2 * hidden)
            a_t = torch.bmm(alpha_t.unsqueeze(1), enc_hiddens).squeeze(1)
            # [(batch, 2 * hidden), (batch, hidden_size)] = (batch, 3 * hidden_size)
            u_t = torch.cat((a_t, hidden_state), 1)
            # (batch, hidden)
            v_t = self.combined_output_projection(u_t)
            combined_output = self.dropout(torch.tanh(v_t))
            combined_outputs.append(combined_output)
        
        combined_outputs = torch.stack(combined_outputs)
        return combined_outputs
    
    # The generate sent masks() function produces a tensor called enc masks.
    # It has shape (batch size, max source sentence length)
    # and contains 1s in positions corresponding to ‘pad’ tokens in the input, and 0s for non-pad tokens.
    # Masks are used during the attention computation.
    def generate_sent_masks(self, enc_hiddens: torch.Tensor, source_lengths: List[int]) -> torch.Tensor:
        enc_masks = torch.zeros(enc_hiddens.size(0), enc_hiddens.size(1), dtype= torch.float)
        for enc_hidden, src_sentence_len in enumerate(source_lengths):
            enc_masks[enc_hidden, src_sentence_len:] = 1
        return enc_masks.to(self.device)
    
    # ЛЕНЬ СУКА
    # step нужен для beam_search
    def step(self, Ybar_t: torch.Tensor,
            dec_state: Tuple[torch.Tensor, torch.Tensor],
            enc_hiddens: torch.Tensor,
            enc_hiddens_proj: torch.Tensor,
            enc_masks: torch.Tensor) -> Tuple[Tuple, torch.Tensor, torch.Tensor]:

        combined_output = None

        # YOUR CODE HERE (~3 Lines)
        dec_state = self.decoder(Ybar_t, dec_state)
        dec_hidden, dec_cell = dec_state[0], dec_state[1]
        e_t = torch.squeeze(torch.bmm(enc_hiddens_proj, torch.unsqueeze(dec_hidden, dim=2)), dim=2)
        # END YOUR CODE

        # Set e_t to -inf where enc_masks has 1
        if enc_masks is not None:
            e_t.data.masked_fill_(enc_masks.bool(), -float('inf'))

        # YOUR CODE HERE (~6 Lines)
        alpha_t = f.softmax(e_t, dim=1)
        a_t = torch.squeeze(torch.bmm(torch.unsqueeze(alpha_t, dim=1), enc_hiddens), dim=1)
        U_t = torch.cat((a_t, dec_hidden), dim=1)
        V_t = self.combined_output_projection(U_t)
        O_t = self.dropout(torch.tanh(V_t))
        # END YOUR CODE

        combined_output = O_t
        return dec_state, combined_output, e_t

    def beam_search(self, src_sent: List[str], beam_size: int=5, max_decoding_time_step: int=70) -> List[Hypothesis]:
        src_sents_var = self.vocab.src.to_input_tensor([src_sent], self.device)

        src_encodings, dec_init_vec = self.encode(src_sents_var, [len(src_sent)])
        src_encodings_att_linear = self.att_projection(src_encodings)

        h_tm1 = dec_init_vec
        att_tm1 = torch.zeros(1, self.hidden_size, device=self.device)

        eos_id = self.vocab.tgt['</s>']

        hypotheses = [['<s>']]
        hyp_scores = torch.zeros(len(hypotheses), dtype=torch.float, device=self.device)
        completed_hypotheses = []

        t = 0
        while len(completed_hypotheses) < beam_size and t < max_decoding_time_step:
            t += 1
            hyp_num = len(hypotheses)

            exp_src_encodings = src_encodings.expand(hyp_num,
                                                     src_encodings.size(1),
                                                     src_encodings.size(2))

            exp_src_encodings_att_linear = src_encodings_att_linear.expand(hyp_num,
                                                                           src_encodings_att_linear.size(1),
                                                                           src_encodings_att_linear.size(2))

            y_tm1 = torch.tensor([self.vocab.tgt[hyp[-1]] for hyp in hypotheses], dtype=torch.long, device=self.device)
            y_t_embed = self.model_embeddings.target(y_tm1)

            x = torch.cat([y_t_embed, att_tm1], dim=-1)

            (h_t, cell_t), att_t, _ = self.step(x, h_tm1,
                                                exp_src_encodings, exp_src_encodings_att_linear, enc_masks=None)

            # log probabilities over target words
            log_p_t = f.log_softmax(self.target_vocab_projection(att_t), dim=-1)

            live_hyp_num = beam_size - len(completed_hypotheses)
            contiuating_hyp_scores = (hyp_scores.unsqueeze(1).expand_as(log_p_t) + log_p_t).view(-1)
            top_cand_hyp_scores, top_cand_hyp_pos = torch.topk(contiuating_hyp_scores, k=live_hyp_num)

            prev_hyp_ids = top_cand_hyp_pos // len(self.vocab.tgt)
            hyp_word_ids = top_cand_hyp_pos % len(self.vocab.tgt)

            new_hypotheses = []
            live_hyp_ids = []
            new_hyp_scores = []

            for prev_hyp_id, hyp_word_id, cand_new_hyp_score in zip(prev_hyp_ids, hyp_word_ids, top_cand_hyp_scores):
                prev_hyp_id = prev_hyp_id.item()
                hyp_word_id = hyp_word_id.item()
                cand_new_hyp_score = cand_new_hyp_score.item()

                hyp_word = self.vocab.tgt.id2word[hyp_word_id]
                new_hyp_sent = hypotheses[prev_hyp_id] + [hyp_word]
                if hyp_word == '</s>':
                    completed_hypotheses.append(Hypothesis(value=new_hyp_sent[1:-1],
                                                           score=cand_new_hyp_score))
                else:
                    new_hypotheses.append(new_hyp_sent)
                    live_hyp_ids.append(prev_hyp_id)
                    new_hyp_scores.append(cand_new_hyp_score)

            if len(completed_hypotheses) == beam_size:
                break

            live_hyp_ids = torch.tensor(live_hyp_ids, dtype=torch.long, device=self.device)
            h_tm1 = (h_t[live_hyp_ids], cell_t[live_hyp_ids])
            att_tm1 = att_t[live_hyp_ids]

            hypotheses = new_hypotheses
            hyp_scores = torch.tensor(new_hyp_scores, dtype=torch.float, device=self.device)

        if len(completed_hypotheses) == 0:
            completed_hypotheses.append(Hypothesis(value=hypotheses[0][1:],
                                                   score=hyp_scores[0].item()))

        completed_hypotheses.sort(key=lambda hyp: hyp.score, reverse=True)

        return completed_hypotheses
    
    def load(model_path: str):
        params = torch.load(model_path, map_location=lambda storage, loc: storage)
        args = params['args']
        model = NMT(vocab=params['vocab'], **args)
        model.load_state_dict(params['state_dict'])

        return model
    
    def save(self, path: str):
        print('save model parameters to [%s]' % path, file=sys.stderr)

        params = {
            'args': dict(
                word_embed_size=self.model_embeddings.word_embed_size,
                hidden_size=self.hidden_size,
                dropout_rate=self.dropout_rate),
            'vocab': self.vocab,
            'state_dict': self.state_dict()
        }

        torch.save(params, path)

<div class='alert alert-block alert-info'>
<b>run.py</b><br>
<b>TRAIN</b>
    <ul>
        <li><i>Что значит cuda?</i> Значит на ГПУ
        <li><i>Как работает yield?</i>
        <li><i>Как запускать?</i> Да тупа вызвать функцию мэйн, только параметры надо передать сразу.
    </ul>
</div>

In [ ]:
def read_corpus(path: str, source: str) -> List[List]:
    data = []
    for line in open(path):
        sentence = nltk.word_tokenize(line)
        if source == 'tgt':
            sentence = ['<s>'] + sentence + ['</s>']
        data.append(sentence)
    
    return data


def batch_iter(data: List[Tuple], batch_size: int, shuffle: bool = False):
    batch_num = math.ceil(len(data) / batch_size)  # количество батчей
    indices = list(range(len(data)))
    
    if shuffle:
        np.random.shuffle(indices)
    
    for i in range(batch_num):
        indices_i = indices[batch_size * i:batch_size * (i + 1)]
        sents = [data[idx] for idx in indices_i]
        
        sents = sorted(sents, key=lambda s: len(s[0]), reverse=True)
        
        src_sents = [s[0] for s in sents]
        tgt_sents = [s[1] for s in sents]
        
        yield src_sents, tgt_sents

In [ ]:
def train(args: Dict):
    # READ PARALLEL CORPORA
    train_data_src = read_corpus(args['--train-src'], source='src')
    train_data_tgt = read_corpus(args['--train-tgt'], source='tgt')
    dev_data_src = read_corpus(args['--dev-src'], source='src')
    dev_data_tgt = read_corpus(args['--dev-tgt'], source='tgt')

    train_data = list(zip(train_data_src, train_data_tgt))
    dev_data = list(zip(dev_data_src, dev_data_tgt))

    # READ PARAMETERS
    train_batch_size = int(args['--batch-size'])
    clip_grad = float(args['--clip-grad'])
    valid_niter = int(args['--valid-niter'])  # perform validation after how many iterations [default: 2000]
    log_every = int(args['--log-every'])
    model_save_path = args['--save-to']

    # READ VOCABULARY
    vocab = Vocab.load(args['--vocab'])

    # INITIALIZE MODEL
    model = NMT(
        word_embed_size=int(args['--embed-size']),
        hidden_size=int(args['--hidden-size']),
        dropout_rate=float(args['--dropout']),
        vocab=vocab
    )
    # Set the module in training mode.
    model.train()
    
    # INITIALIZE PARAMETERS
    uniform_init = float(args['--uniform-init'])  # uniformly initialize all parameters [default: 0.1]
    if np.abs(uniform_init) > 0.:
        print('uniformly initialize parameters [-%f, +%f]' % (uniform_init, uniform_init), file=sys.stderr)
        for p in model.parameters():
            p.data.uniform_(-uniform_init, uniform_init)

    ##### ВОТ ТУТ НЕ ПОНЕЛ #####
    vocab_mask = torch.ones(len(vocab.tgt))
    vocab_mask[vocab.tgt['<pad>']] = 0

    device = torch.device("cuda:0" if args['--cuda'] else "cpu")
    print('use device: %s' % device, file=sys.stderr)
    # Moves and/or casts the parameters and buffers.
    model = model.to(device)

    optimizer = torch.optim.Adam(model.parameters(), lr=float(args['--lr']))

    # INITIALIZE VARIABLES DESCRIBING TRAINING
    num_trial = 0
    train_iter = patience = cum_loss = report_loss = cum_tgt_words = report_tgt_words = 0
    cum_examples = report_examples = epoch = valid_num = 0
    hist_valid_scores = []
    train_time = begin_time = time.time()
    print('begin Maximum Likelihood training')
    
    # TRAIN
    while True:
        epoch += 1
        for src_sents, tgt_sents in batch_iter(train_data, batch_size=train_batch_size, shuffle=True):
            # ОБУЧЕНИЕ
            train_iter += 1

            # initialize gradients
            optimizer.zero_grad()
            batch_size = len(src_sents)
            # calc losses for every sentence
            example_losses = -model(src_sents, tgt_sents)  # (batch_size,)
            # sum it up
            batch_loss = example_losses.sum()
            # average loss
            loss = batch_loss / batch_size
            # CALCULATE GRADIENT
            loss.backward()
            # clip gradient
            grad_norm = torch.nn.utils.clip_grad_norm_(model.parameters(), clip_grad)
            # UPDATE PARAMETERS
            optimizer.step()

            # ЛОГИ
            batch_losses_val = batch_loss.item()
            report_loss += batch_losses_val
            cum_loss += batch_losses_val

            tgt_words_num_to_predict = sum(len(s[1:]) for s in tgt_sents)  # omitting leading `<s>`
            report_tgt_words += tgt_words_num_to_predict
            cum_tgt_words += tgt_words_num_to_predict
            report_examples += batch_size
            cum_examples += batch_size
            
            if train_iter % log_every == 0:
                # print('epoch %d, iter %d, avg. loss %.2f, avg. ppl %.2f ' \
                #       'cum. examples %d, speed %.2f words/sec, time elapsed %.2f sec' % (
                #     epoch, train_iter, report_loss / report_examples, math.exp(report_loss / report_tgt_words),
                #     cum_examples, report_tgt_words / (time.time() - train_time), time.time() - begin_time
                # ))
                wandb.log({"loss":report_loss / report_examples, "ppl": math.exp(report_loss / report_tgt_words)})

                train_time = time.time()
                report_loss = report_tgt_words = report_examples = 0.
                
            # ВАЛИДАЦИЯ
            if train_iter % valid_niter == 0:
                print('epoch %d, iter %d, cum. loss %.2f, cum. ppl %.2f cum. examples %d' % (
                    epoch, train_iter, cum_loss / cum_examples, np.exp(cum_loss / cum_tgt_words), cum_examples
                ))

                cum_loss = cum_examples = cum_tgt_words = 0.
                valid_num += 1

                print('begin validation ...')

                # compute dev. ppl and bleu
                dev_ppl = evaluate_ppl(model, dev_data, batch_size=128)   # dev batch size can be a bit larger
                valid_metric = -dev_ppl

                print('validation: iter %d, dev. ppl %f' % (train_iter, dev_ppl))

                is_better = len(hist_valid_scores) == 0 or valid_metric > max(hist_valid_scores)
                hist_valid_scores.append(valid_metric)

                if is_better:
                    patience = 0
                    print('save currently the best model to [%s]' % model_save_path)
                    model.save(model_save_path)

                    # also save the optimizers' state
                    torch.save(optimizer.state_dict(), model_save_path + '.optim')
                elif patience < int(args['--patience']):
                    patience += 1
                    print('hit patience %d' % patience, file=sys.stderr)

                    if patience == int(args['--patience']):
                        num_trial += 1
                        print('hit #%d trial' % num_trial, file=sys.stderr)
                        if num_trial == int(args['--max-num-trial']):
                            print('early stop!', file=sys.stderr)
                            exit(0)

                        # decay lr, and restore from previously best checkpoint
                        lr = optimizer.param_groups[0]['lr'] * float(args['--lr-decay'])
                        print('load previously best model and decay learning rate to %f' % lr)

                        # LOAD
                        params = torch.load(model_save_path, map_location=lambda storage, loc: storage)
                        model.load_state_dict(params['state_dict'])
                        model = model.to(device)

                        print('restore parameters of the optimizers')
                        optimizer.load_state_dict(torch.load(model_save_path + '.optim'))

                        # set new lr
                        for param_group in optimizer.param_groups:
                            param_group['lr'] = lr

                        # reset patience
                        patience = 0
                if epoch == int(args['--max-epoch']):
                    print('reached maximum number of epochs!')
                    exit(0)

In [ ]:
def evaluate_ppl(model, dev_data, batch_size=32):
    # save the previous mode and turn to evaluation mode
    was_training = model.training
    model.eval()

    cum_loss = 0.
    cum_tgt_words = 0.

    # no_grad() signals backend to throw away all gradients
    with torch.no_grad():
        for src_sents, tgt_sents in batch_iter(dev_data, batch_size):
            # __call__ == forward
            loss = -model(src_sents, tgt_sents).sum()

            cum_loss += loss.item()
            tgt_word_num_to_predict = sum(len(s[1:]) for s in tgt_sents)  # omitting leading `<s>`
            cum_tgt_words += tgt_word_num_to_predict

        ppl = np.exp(cum_loss / cum_tgt_words)

    # return previous mode
    if was_training:
        model.train()

    return ppl

In [ ]:
def compute_corpus_level_bleu_score(references: List[List[str]], hypotheses: List[Hypothesis]) -> float:
    # cut off <s> and </s>
    if references[0][0] == '<s>':
        references = [ref[1:-1] for ref in references]
    bleu_score = corpus_bleu([[ref] for ref in references],
                             [hyp.value for hyp in hypotheses])
    return bleu_score

<div class='alert alert-block alert-info'>
<b>run.py</b><br>
<b>TEST</b>
    <ul>
        <li><i>Что значит cuda?</i> Значит на ГПУ
        <li><i>Как работает yield?</i>
        <li><i>Как запускать?</i> Да тупа вызвать функцию мэйн, только параметры надо передать сразу.
    </ul>
</div>

In [ ]:
# TEST MODEL
# тут пример создания новой модели
def decode(args: Dict[str, str]):
    print("load test source sentences from [{}]".format(args['TEST_SOURCE_FILE']))
    test_data_src = read_corpus(args['TEST_SOURCE_FILE'], source='src')
    if args['TEST_TARGET_FILE']:
        print("load test target sentences from [{}]".format(args['TEST_TARGET_FILE']))
        test_data_tgt = read_corpus(args['TEST_TARGET_FILE'], source='tgt')

    # we create new model
    print("load model from {}".format(args['MODEL_PATH']))
    model = NMT.load(args['MODEL_PATH'])

    if args['--cuda']:
        model = model.to(torch.device("cuda:0"))

    hypotheses = beam_search(model, test_data_src,
                             beam_size=int(args['--beam-size']),
                             max_decoding_time_step=int(args['--max-decoding-time-step']))

    if args['TEST_TARGET_FILE']:
        top_hypotheses = [hyps[0] for hyps in hypotheses]
        bleu_score = compute_corpus_level_bleu_score(test_data_tgt, top_hypotheses)
        print('Corpus BLEU: {}'.format(bleu_score * 100))

    with open(args['OUTPUT_FILE'], 'w') as f:
        for src_sent, hyps in zip(test_data_src, hypotheses):
            top_hyp = hyps[0]
            hyp_sent = ' '.join(top_hyp.value)
            f.write(hyp_sent + '\n')

In [ ]:
def beam_search(model: NMT, test_data_src: List[List[str]], beam_size: int, max_decoding_time_step: int) -> List[List[Hypothesis]]:
    training_mode = model.training  # returns True if model was in a training mode
    model.eval()  # turn to evaluation mode

    # generate hypotheses
    hypotheses = []
    
    # Disabling gradient calculation is useful for inference,
    # when you are sure that you will not call Tensor.backward().
    # It will reduce memory consumption for computations that would otherwise have requires_grad=True.
    with torch.no_grad():
        for src_sent in tqdm(test_data_src, desc='Decoding', file=sys.stdout):
            # call internal beam_search
            example_hyps = model.beam_search(
                src_sent, beam_size=beam_size, max_decoding_time_step=max_decoding_time_step
            )
            
            # add a new generated hypotheses for current source sentence
            hypotheses.append(example_hyps)

    # turn the mode back
    if training_mode:
        model.train(training_mode)

    return hypotheses

In [ ]:
# вызов train_local

# !python main train
# --train-src=/Users/ainurmukh/nn_course/a5_public/en_es_data/train.es
# --train-tgt=/Users/ainurmukh/nn_course/a5_public/en_es_data/train.en
# --dev-src=/Users/ainurmukh/nn_course/a5_public/en_es_data/dev.es
# --dev-tgt=/Users/ainurmukh/nn_course/a5_public/en_es_data/dev.en
# --vocab=vocab.json

In [ ]:
# ВОТ ТАКАЯ ШТУКА В МЭЙНЕ

def main(args):
    # читаем аргументы из ввода
    # args = docopt(__doc__)

    seed = int(args['--seed'])
    torch.manual_seed(seed)
    if args['--cuda']:
        torch.cuda.manual_seed(seed)
    np.random.seed(seed * 13 // 7)

    if args['train']:
        train(args)
    elif args['decode']:
        decode(args)
    else:
        raise RuntimeError('invalid run mode')

In [ ]:
train_args = {
 '--batch-size': '32',
 '--beam-size': '5',
 '--clip-grad': '5.0',
 '--cuda': True,
 '--dev-src': '/content/gdrive/MyDrive/Colab/a4/en_es_data/dev.es',
 '--dev-tgt': '/content/gdrive/MyDrive/Colab/a4/en_es_data/dev.en',
 '--dropout': '0.3',
 '--embed-size': '256',
 '--help': False,
 '--hidden-size': '256',
 '--input-feed': False,
 '--log-every': '10',
 '--lr': '0.001',
 '--lr-decay': '0.5',
 '--max-decoding-time-step': '70',
 '--max-epoch': '30',
 '--max-num-trial': '5',
 '--patience': '5',
 '--sample-size': '5',
 '--save-to': '/content/gdrive/MyDrive/Colab/model.bin',
 '--seed': '0',
 '--train-src': '/content/gdrive/MyDrive/Colab/a4/en_es_data/train.es',
 '--train-tgt': '/content/gdrive/MyDrive/Colab/a4/en_es_data/train.en',
 '--uniform-init': '0.1',
 '--valid-niter': '2000',
 '--vocab': '/content/gdrive/MyDrive/Colab/a4/vocab.json',
 'MODEL_PATH': None,
 'OUTPUT_FILE': None,
 'TEST_SOURCE_FILE': None,
 'TEST_TARGET_FILE': None,
 'decode': False,
 'train': True}

In [ ]:
wandb.init(project="test-drive", config={
    "learning_rate": 0.001,
    "dropout": 0.3,
    "architecture": "NMT_no_decoder",
})
config = wandb.config

wandb: Currently logged in as: ainurmukh (use `wandb login --relogin` to force relogin)


In [ ]:
main(train_args)

uniformly initialize parameters [-0.100000, +0.100000]
use device: cuda:0


begin Maximum Likelihood training
epoch 1, iter 2000, cum. loss 96.27, cum. ppl 107.68 cum. examples 64000
begin validation ...
validation: iter 2000, dev. ppl 33.845637
save currently the best model to [/content/gdrive/MyDrive/Colab/model.bin]


save model parameters to [/content/gdrive/MyDrive/Colab/model.bin]


epoch 1, iter 4000, cum. loss 67.31, cum. ppl 26.30 cum. examples 64000
begin validation ...
validation: iter 4000, dev. ppl 16.583964
save currently the best model to [/content/gdrive/MyDrive/Colab/model.bin]


save model parameters to [/content/gdrive/MyDrive/Colab/model.bin]


...

epoch 15, iter 96000, cum. loss 24.46, cum. ppl 3.29 cum. examples 63977
begin validation ...
validation: iter 96000, dev. ppl 7.238782
load previously best model and decay learning rate to 0.000031


hit patience 5
hit #5 trial
early stop!


restore parameters of the optimizers
epoch 15, iter 98000, cum. loss 25.15, cum. ppl 3.38 cum. examples 64000
begin validation ...
validation: iter 98000, dev. ppl 7.193303


hit patience 1


KeyboardInterrupt: ignored

<div class='alert alert-block alert-info'>
<b>ЗАПУСК</b>
    <ul>
        <li><i>Откуда берутся параметры, которые мы не указываем</i>
        <li><b><i>Как запускать с определенного места?</i></b>
    </ul>
</div>

In [ ]:
test_args = {
 '--batch-size': '32',
 '--beam-size': '5',
 '--clip-grad': '5.0',
 '--cuda': True,
 '--dev-src': None,
 '--dev-tgt': None,
 '--dropout': '0.3',
 '--embed-size': '256',
 '--help': False,
 '--hidden-size': '256',
 '--input-feed': False,
 '--log-every': '10',
 '--lr': '0.001',
 '--lr-decay': '0.5',
 '--max-decoding-time-step': '70',
 '--max-epoch': '30',
 '--max-num-trial': '5',
 '--patience': '5',
 '--sample-size': '5',
 '--save-to': '/content/gdrive/MyDrive/Colab/model.bin',
 '--seed': '0',
 '--train-src': None,
 '--train-tgt': None,
 '--uniform-init': '0.1',
 '--valid-niter': '2000',
 '--vocab': None,
 'MODEL_PATH': '/content/gdrive/MyDrive/Colab/model.bin',
 'OUTPUT_FILE': '/content/gdrive/MyDrive/Colab/test_outputs.txt',
 'TEST_SOURCE_FILE': '/content/gdrive/MyDrive/Colab/a4/en_es_data/test.es',
 'TEST_TARGET_FILE': '/content/gdrive/MyDrive/Colab/a4/en_es_data/test.en',
 'decode': True,
 'train': False}


In [ ]:
main(test_args)

load test source sentences from [/content/gdrive/MyDrive/Colab/a4/en_es_data/test.es]
load test target sentences from [/content/gdrive/MyDrive/Colab/a4/en_es_data/test.en]
load model from /content/gdrive/MyDrive/Colab/model.bin



Corpus BLEU: 35.77854110593709


In [ ]:
from tqdm.notebook import tqdm